In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os.path as op
import pandas as pd

chromsizes = []
with open('/Users/pete/projects/negspy/negspy/data/b37/chromInfo.txt') as f:
    for line in f:
        chromsizes += [line.strip().split('\t')]

chromsizes_hg19 = []
with open('/Users/pete/projects/negspy/negspy/data/hg19/chromInfo.txt') as f:
    for line in f:
        chromsizes_hg19 += [line.strip().split('\t')]

In [3]:
%%time

filename = '/Users/pete/projects/resgen-server/media/s3/resgen-test/test.resgen.io/QzV6Qhx7QYaIP9nEznkVig/HG002_SVs_Tier1_v0.6.bed..'
t = pd.read_csv(filename, header=None, delimiter="\t")

CPU times: user 23.1 ms, sys: 9.5 ms, total: 32.6 ms
Wall time: 1.24 s


In [4]:
import numpy as np
def chromsizes_array_to_series(chromsizes):
    """
    Convert an array of [[chrname, size]...] values to a series
    indexed by chrname with size values
    """
    chrnames = [c[0] for c in chromsizes]
    chrvalues = [c[1] for c in chromsizes]

    return pd.Series(np.array([int(c) for c in chrvalues]), index=chrnames)

In [5]:
cs_df = chromsizes_array_to_series(chromsizes)
css = cs_df.cumsum().shift().fillna(0).to_dict()

In [6]:
cs_df_hg19 = chromsizes_array_to_series(chromsizes_hg19)
css_hg19 = cs_df_hg19.cumsum().shift().fillna(0).to_dict()

## Load tbi bedfile

In [29]:
filename ="/Users/pete/projects/paper-data/zook-2020/hg19.repeats.trf.bed.gz"
index_filename = "/Users/pete/projects/paper-data/zook-2020/hg19.repeats.trf.bed.gz.tbi.12"

In [37]:
import pysam
tb = pysam.TabixFile(filename, index=index_filename)

In [76]:
%%time

import clodius.tiles.bedfile as ctb
import clodius.tiles.tabix as ctt

tbx_index = ctt.load_tbi_idx(index_filename)
tsinfo = ctb.tileset_info(filename, cs_df_hg19, index_filename)
t = ctb.single_indexed_tile(filename, index_filename, cs_df_hg19, tsinfo, 6, 0, tbx_index)
print("len", len(t))

len 19854
CPU times: user 728 ms, sys: 103 ms, total: 831 ms
Wall time: 862 ms


In [63]:
len(t)

59

In [8]:
import clodius.tiles.bedfile as ctbf

tsinfo = ctbf.tileset_info(filename, cs_df)
tsinfo

{'max_width': 3095693981,
 'max_zoom': 31,
 'chromsizes': [['1', 249250621],
  ['2', 243199373],
  ['3', 198022430],
  ['4', 191154276],
  ['5', 180915260],
  ['6', 171115067],
  ['7', 159138663],
  ['8', 146364022],
  ['9', 141213431],
  ['10', 135534747],
  ['11', 135006516],
  ['12', 133851895],
  ['13', 115169878],
  ['14', 107349540],
  ['15', 102531392],
  ['16', 90354753],
  ['17', 81195210],
  ['18', 78077248],
  ['19', 59128983],
  ['20', 63025520],
  ['21', 48129895],
  ['22', 51304566],
  ['X', 155270560],
  ['Y', 59373566],
  ['MT', 16569]],
 'min_pos': [0],
 'max_pos': [3095693981]}

In [9]:
%%time

print(ctbf.single_tile(filename, cs_df, tsinfo, 17, 35))
print(ctbf.single_tile(filename, cs_df, tsinfo, 18, 70))
print(ctbf.single_tile(filename, cs_df, tsinfo, 18, 71))
# x = ctbf.single_tile(filename, cs_df, tsinfo, 15, 9331)


[{'uid': 0, 'xStart': 834131.0, 'xEnd': 843115.0, 'chrOffset': 0.0, 'importance': 0.5233906436680045, 'fields': ['1', 834131, 843115]}, {'uid': 1, 'xStart': 846279.0, 'xEnd': 868342.0, 'chrOffset': 0.0, 'importance': 0.11077413746686304, 'fields': ['1', 846279, 868342]}]
[{'uid': 0, 'xStart': 834131.0, 'xEnd': 843115.0, 'chrOffset': 0.0, 'importance': 0.32331359754307, 'fields': ['1', 834131, 843115]}]
[{'uid': 0, 'xStart': 834131.0, 'xEnd': 843115.0, 'chrOffset': 0.0, 'importance': 0.1821769631922241, 'fields': ['1', 834131, 843115]}, {'uid': 1, 'xStart': 846279.0, 'xEnd': 868342.0, 'chrOffset': 0.0, 'importance': 0.4187216749401593, 'fields': ['1', 846279, 868342]}]
CPU times: user 136 ms, sys: 32.5 ms, total: 168 ms
Wall time: 196 ms


In [108]:
t['ix'] = t.index

In [122]:
t.tail().apply(lambda x: x['ix'], axis=1)

34825    34825
34826    34826
34827    34827
34828    34828
34829    34829
dtype: int64

In [46]:
%%time

s = 0
for item in t.iterrows():
    s += item[1][1]


CPU times: user 2.42 s, sys: 27.2 ms, total: 2.45 s
Wall time: 2.49 s


In [48]:
%%time

t.sort_values(by=[0,1])

CPU times: user 23.1 ms, sys: 11.5 ms, total: 34.6 ms
Wall time: 54.6 ms


,0,1,2
0,1,834131,843115
1,1,846279,868342
2,1,871339,871544
3,1,871891,883971
4,1,884192,911068
5,1,914552,919891
6,1,920092,924011
7,1,924800,928212
8,1,928587,934027
9,1,934279,957867


## GIAB indexed bedfile

In [60]:
filename = op.join(
    "/Users/pete/projects/resgen-server/media/https/"
    "dl.dnanex.us/F/D/qq3pZ3X5Q5p4Pbzzgk6FyzqVbKk02KqbYQZ1p0gP/hg19.segdups%2Bselfchain.bed.gz.."
)

In [6]:
filename

'/Users/pete/projects/resgen-server/media/https/dl.dnanex.us/F/D/qq3pZ3X5Q5p4Pbzzgk6FyzqVbKk02KqbYQZ1p0gP/hg19.segdups%2Bselfchain.bed.gz..'

In [7]:
op.exists(filename)

True

In [22]:
import clodius.tiles.bedfile as ctb

if filename[-2:] == '..':
    index_filename = filename[:-2] + '.tbi..'
else:
    index_filename = filename + '.tbi'

op.exists(index_filename)
#ctb.single_indexed_tile()

True

In [39]:
import tabix
tb = tabix.open(filename, fnidx='xxx')
tb.query('chr1', 0, 40000)

TabixError: query failed

In [42]:
import tabix

filename='/Users/pete/Downloads/hg19.segdups+selfchain.bed.gz'
tb = tabix.open(filename)
tb.query('chr1', 0, 20000)

In [40]:
tabix.__file__

'/Users/pete/miniconda3/envs/cenv4/lib/python3.7/site-packages/tabix.cpython-37m-darwin.so'

In [ ]:
x="https://dl.dnanex.us/F/D/qq3pZ3X5Q5p4Pbzzgk6FyzqVbKk02KqbYQZ1p0gP/hg19.segdups%2Bselfchain.bed.gz"

In [43]:
pd.read_csv(filename, header=None, delimiter='\t')

,0,1,2,3,4,5
0,chr1,10000,19844,"segdup:chrX:155,249,882-155,260,239",-,983
1,chr1,10000,19844,"segdup:chrY:59,352,888-59,363,245",-,983
2,chr1,10000,20818,"segdup:chr12:84,887-95,696",-,982
3,chr1,10000,39170,"selfchain:chr12:64,520-95,624",-,897
4,chr1,10000,87112,"segdup:chr15:102,446,356-102,521,392",-,993
5,chr1,10464,19844,"selfchain:chrX:155,249,882-155,259,196",-,899
6,chr1,10464,19844,"selfchain:chrY:59,352,888-59,362,202",-,899
7,chr1,10464,40733,"segdup:chr2:114,330,298-114,359,986",-,988
8,chr1,10485,19844,"segdup:chr16:60,427-69,533",+,987
9,chr1,10485,40733,"segdup:chr9:10,844-40,515",+,994


In [57]:
import clodius.tiles.bedfile as ctb

ctb.tileset_info(filename, cs_df_hg19)

{'max_width': 3137161264,
 'max_zoom': 31,
 'chromsizes': [['chr1', 249250621],
  ['chr2', 243199373],
  ['chr3', 198022430],
  ['chr4', 191154276],
  ['chr5', 180915260],
  ['chr6', 171115067],
  ['chr7', 159138663],
  ['chr8', 146364022],
  ['chr9', 141213431],
  ['chr10', 135534747],
  ['chr11', 135006516],
  ['chr12', 133851895],
  ['chr13', 115169878],
  ['chr14', 107349540],
  ['chr15', 102531392],
  ['chr16', 90354753],
  ['chr17', 81195210],
  ['chr18', 78077248],
  ['chr19', 59128983],
  ['chr20', 63025520],
  ['chr21', 48129895],
  ['chr22', 51304566],
  ['chrX', 155270560],
  ['chrY', 59373566],
  ['chrM', 16571],
  ['chr6_ssto_hap7', 4928567],
  ['chr6_mcf_hap5', 4833398],
  ['chr6_cox_hap2', 4795371],
  ['chr6_mann_hap4', 4683263],
  ['chr6_apd_hap1', 4622290],
  ['chr6_qbl_hap6', 4611984],
  ['chr6_dbb_hap3', 4610396],
  ['chr17_ctg5_hap1', 1680828],
  ['chr4_ctg9_hap1', 590426],
  ['chr1_gl000192_random', 547496],
  ['chrUn_gl000225', 211173],
  ['chr4_gl000194_random', 

In [59]:
filename

'/Users/pete/Downloads/hg19.segdups+selfchain.bed.gz'

In [178]:
%%time

tsinfo = ctb.tileset_info(filename, cs_df_hg19)
# t = ctb.single_tile(filename, cs_df_hg19, tsinfo, 0, 0)
t = ctb.tiles(filename, ["x.0.0"], cs_df_hg19)

CPU times: user 265 ms, sys: 35.1 ms, total: 300 ms
Wall time: 307 ms


In [177]:
%%time

ctb.single_tile(filename, cs_df_hg19, tsinfo, 0, 0)

CPU times: user 305 ms, sys: 39.4 ms, total: 344 ms
Wall time: 399 ms


[{'uid': 1267063,
  'xStart': 1680301403.0,
  'xEnd': 1680302603.0,
  'chrOffset': 1539159712.0,
  'importance': 0.6204242729787587,
  'fields': ['chr9',
   141141691,
   141142891,
   'selfchain:chr7:56,934,786-56,936,016',
   '-',
   430]},
 {'uid': 1185858,
  'xStart': 1391782060.0,
  'xEnd': 1391782456.0,
  'chrOffset': 1233657027.0,
  'importance': 0.38567169425316605,
  'fields': ['chr7',
   158125033,
   158125429,
   'selfchain:chr7:158,134,014-158,134,409',
   '+',
   510]},
 {'uid': 399481,
  'xStart': 1930394954.0,
  'xEnd': 1930395430.0,
  'chrOffset': 1815907890.0,
  'importance': 0.6889503682619283,
  'fields': ['chr11',
   114487064,
   114487540,
   'selfchain:chr11:114,494,608-114,495,083',
   '+',
   131]},
 {'uid': 1264151,
  'xStart': 1679619502.0,
  'xEnd': 1679619590.0,
  'chrOffset': 1539159712.0,
  'importance': 0.32498546360249847,
  'fields': ['chr9',
   140459790,
   140459878,
   'selfchain:chr9:140,460,353-140,460,468',
   '+',
   848]},
 {'uid': 675967,
  

In [136]:
%%time

leftix = np.searchsorted(t['rows']['xEnd'], 20000, 'right', t['sortedEnds'])[0]
rightix = np.searchsorted(t['rows']['xStart'], 20000, 'right', t['sortedStarts'])[0]

leftix, rightix

CPU times: user 679 µs, sys: 47 µs, total: 726 µs
Wall time: 694 µs


(20, 30)

In [179]:
t

[('x.0.0',
  [{'uid': 554821,
    'xStart': 2408381758.0,
    'xEnd': 2408381865.0,
    'chrOffset': 2307285719.0,
    'importance': 0.9577033307490938,
    'fields': ['chr15',
     101096039,
     101096146,
     'selfchain:chr15:101,098,503-101,098,608',
     '+',
     652]},
   {'uid': 477301,
    'xStart': 2197734983.0,
    'xEnd': 2197735201.0,
    'chrOffset': 2084766301.0,
    'importance': 0.4329909285355267,
    'fields': ['chr13',
     112968682,
     112968900,
     'selfchain:chr13:112,950,396-112,950,577',
     '+',
     223]},
   {'uid': 619833,
    'xStart': 2525955424.0,
    'xEnd': 2525955601.0,
    'chrOffset': 2500171864.0,
    'importance': 0.09657697192735393,
    'fields': ['chr17',
     25783560,
     25783737,
     'selfchain:chr17:25,290,272-25,290,427',
     '+',
     662]},
   {'uid': 199011,
    'xStart': 143286631.0,
    'xEnd': 143286872.0,
    'chrOffset': 0.0,
    'importance': 0.8916870896716564,
    'fields': ['chr1',
     143286631,
     143286872,
  

In [139]:
t['sortedEnds']

array([     14,      17,      16, ..., 1276127, 1276134,  650053])

In [145]:
%%time

x = t['rows']

t1 = x[x['xStart'] <= 30000]
t2 = t1[t1['xEnd'] >= 20000]

CPU times: user 3.93 ms, sys: 1.39 ms, total: 5.31 ms
Wall time: 3.71 ms
